# Работа с данными для корректного запуска модели #

In [389]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

path = '/Users/maricolada/Downloads/elbrus/_data.csv'
df = pd.read_csv(path)

df.head(5)

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,...,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",...,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",...,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615


In [379]:
#Делаем проверку на наличие дублей
duplicates = df.duplicated()
duplicates_df = duplicates.any()
duplicates_df

False

### Удаление избыточных признаков ###

В результате первичного анализа мы решили, что текущий датафрейм содержит много избыточной информации, которая является слабым фактором для формирования спроса и предложения, а соответственно и ценообразования. Мы выделили следующие признаки в качестве избыточных, при условии выделения нескольких новых:

- **"Тип"** - его отсутствие уравновешивается признаками "Площадь, м2" и количеством санузлов ("Санузлы").

- **"Адрес"** - его отсутствие уравновешивается признаками "Метро" и **новым признаком "Время до метро пешком"**.

- **"Телефоны**, **"Описание**, **"Ссылка на объявление"** - эти признаки носят информационную функцию и обсуловлены всеми остальными признаками: то есть они нужны уже после того, как потенциальный арендатор остановится на каком-либо варианте аренды или примет окончательное решение.

- **"Окна"** - этот признак уравновешивается признаками "Балкон" и **новым признаком "Этаж"**.
  
- **"Дополнительно"** - данный признак мы заменяем на **пять новых признаков**, которые являются более конкретными и поддаются числовому эквиваленту на основе их наличия или отсутствия - **"Мебель в комнатах", "Мебель на кухне", "Холодильник", "Стиральная машина", "Интернет"**.

- **"Мусоропровод"** - этот признак не является решающим фактором при формировании цены, поскольку в обществе есть два полярных мнения от "Я обожаю мусоропроводы, потому что не надо никуда выходить" до "Никогда не буду жить в доме с мусоропроводом". При этом в составе первичных данных очень много пропусков, а заменять на рандомные "да" или "нет", чтобы заполнить пробелы - слишком рисковано для объективности расчета.

In [391]:
#удаляем лишние признаки, кроме "Дом" и "Дополнительно" - они понадобятся нам дальше для создания новых признаков
df = df.drop(columns=['Тип', 'Адрес', 'Телефоны', 'Описание', 'Площадь комнат, м2', 'Окна', 'Название ЖК', 'Серия дома', 'Мусоропровод', 'Ссылка на объявление'])
df.head(5)

,Unnamed: 0,ID объявления,Количество комнат,Метро,"Площадь, м2",Дом,Парковка,Цена,Ремонт,Балкон,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт
0,0,271271157,4,м. Смоленская (9 мин пешком),200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.0,"Пасс (4), Груз (1)"
1,1,271634126,4,м. Смоленская (8 мин пешком),198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.5,"Пасс (1), Груз (1)"
2,2,271173086,"4, Оба варианта",м. Смоленская (7 мин пешком),200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,NaN,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1)
3,3,272197456,"4, Оба варианта",м. Смоленская (3 мин пешком),170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,NaN,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1)
4,4,273614615,2,м. Арбатская (7 мин пешком),58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,NaN,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.9,"Пасс (1), Груз (1)"


### Добавление новых признаков ###

Мы посчитали важным для точности работы модели добавить новые признаки:

- **"Время до метро пешком"** - вместо признака "Адрес", при этом перенеся часть информации из признака "Метро".
  
- **"Этаж", "Тип дома"** - сейчас эти признаки объединены в единый признак "Дом", однако являются разнородной информацией, которая влияет на ценообразование.

- **"Мебель в комнатах", "Мебель на кухне", "Холодильник", "Стиральная машина", "Интернет"** - вместо единого признака "Дополнительно".

In [393]:
# Добавляем новые признаки

#"Время до метро пешком" и сразу заполняем новую колонку

df['Время до метро пешком'] = df['Метро'].str.extract(r'(\d+)\sмин')[0]
df['Время до метро пешком'] = pd.to_numeric(df['Время до метро пешком'], errors='coerce')

mean_min = df['Время до метро пешком'].mean()

df['Время до метро пешком'] = df['Время до метро пешком'].fillna(mean_min)

df['Время до метро пешком'] = df['Время до метро пешком'].astype(int)

metro_index = df.columns.get_loc('Метро')
df = df[['Unnamed: 0', 'ID  объявления', 'Количество комнат', 'Метро', 'Время до метро пешком', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Ремонт', 'Балкон', 'Санузел', 'Можно с детьми/животными',	'Дополнительно', 'Высота потолков, м',	'Лифт']]

#"Этаж" и "Тип дома" вместо "Дом" и сразу заполняем новые колонки

df[['Этаж', 'Тип дома']] = df['Дом'].str.split(', ', expand=True)
df['Этаж'] = df['Этаж'].str.split('/').str[0].astype(int)
df.drop('Дом', axis=1, inplace=True)

col = df.columns.tolist()
index = col.index('Площадь, м2')  # индекс колонки, после которой выстраиваем

# "Мебель в комнатах", "Мебель на кухне", "Холодильник", "Стиральная машина", "Интернет" вместо единого признака "Дополнительно"
# и сразу заполнение в зависимости от наличия (1) или отсутствия (0)

split_columns = df['Дополнительно'].str.split(', ', expand=True)

column_names = ['Мебель в комнатах', 'Мебель на кухне', 'Холодильник', 'Стиральная машина', 'Интернет']

for col, name in zip(range(split_columns.shape[1]), column_names):
    df1[name] = split_columns[col]

df.loc[df['Дополнительно'].str.contains('Мебель в комнатах', na=False), 'Мебель в комнатах'] = 1
df.loc[~df['Дополнительно'].str.contains('Мебель в комнатах', na=True), 'Мебель в комнатах'] = 0
df.loc[df['Дополнительно'].str.contains('Мебель на кухне', na=False), 'Мебель на кухне'] = 1
df.loc[~df['Дополнительно'].str.contains('Мебель на кухне', na=True), 'Мебель на кухне'] = 0
df.loc[df['Дополнительно'].str.contains('Холодильник', na=False), 'Холодильник'] = 1
df.loc[~df['Дополнительно'].str.contains('Холодильник', na=True), 'Холодильник'] = 0
df.loc[df['Дополнительно'].str.contains('Стиральная машина', na=False), 'Стиральная машина'] = 1
df.loc[~df['Дополнительно'].str.contains('Стиральная машина', na=True), 'Стиральная машина'] = 0
df.loc[df['Дополнительно'].str.contains('Интернет', na=False), 'Интернет'] = 1
df.loc[~df['Дополнительно'].str.contains('Интернет', na=True), 'Интернет'] = 0

df.drop('Дополнительно', axis=1, inplace=True)

col2 = df.columns.tolist()
index = col2.index('Можно с детьми/животными')  # индекс колонки, после которой выстраиваем
df = df[[*col2[:index], 'Мебель в комнатах', 'Мебель на кухне', 'Холодильник', 'Стиральная машина', 'Интернет', *col2[index:]]]

df.head(5)

,Unnamed: 0,ID объявления,Количество комнат,Метро,Время до метро пешком,"Площадь, м2",Парковка,Цена,Ремонт,Балкон,...,Можно с детьми/животными,"Высота потолков, м",Лифт,Этаж,Тип дома,Мебель в комнатах,Мебель на кухне,Холодильник,Стиральная машина,Интернет
0,0,271271157,4,м. Смоленская (9 мин пешком),9,200.0/20.0,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,...,"Можно с детьми, Можно с животными",3.0,"Пасс (4), Груз (1)",5,Монолитный,1.0,1.0,1.0,1.0,1.0
1,1,271634126,4,м. Смоленская (8 мин пешком),8,198.0/95.0/18.0,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,...,Можно с детьми,3.5,"Пасс (1), Груз (1)",5,Монолитно-кирпичный,1.0,1.0,1.0,1.0,1.0
2,2,271173086,"4, Оба варианта",м. Смоленская (7 мин пешком),7,200.0/116.0/4.0,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,NaN,...,Можно с детьми,3.2,Пасс (1),5,None,1.0,1.0,1.0,1.0,1.0
3,3,272197456,"4, Оба варианта",м. Смоленская (3 мин пешком),3,170.0/95.0/17.0,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,NaN,...,Можно с животными,3.2,Пасс (1),5,None,1.0,1.0,1.0,1.0,1.0
4,4,273614615,2,м. Арбатская (7 мин пешком),7,58.0/38.0/5.0,NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,NaN,...,NaN,3.9,"Пасс (1), Груз (1)",12,Панельный,1.0,1.0,1.0,1.0,1.0


### Очищение признаков от мусорных данных ###

- **'Количество комнат'** - необходимо оставить только количественный показатель - число.

- **'Метро'** - необходимо оставить только название станции.
  
- **'Площадь, м2'** - необходимо оставить только первое число типа float - показатель общей площади квартиры.
  
- **'Цена'** - необходимо оставить только первое число типа float - показатель стоимости аренды в месяц - основной ориентир для арендаторов. Залог - это уже устоявшаяся практика и воспринимается как само собой разумеется - 100% от стоимости месячной аренды в первый месяц.

In [395]:
# Очищаем признак "Количество комнат" от мусора

df['Количество комнат'] = df['Количество комнат'].astype(str).str.extract(r'(\d+)')[0]
df['Количество комнат'] = pd.to_numeric(df['Количество комнат'], errors='coerce')
df['Количество комнат'] = df['Количество комнат'].fillna(1)
df['Количество комнат'] = df['Количество комнат'].astype(int)

# Очищаем признак "Метро" от мусора

def metro(s):
    s=s[3:]
    for i in range(0,len(s)):
        if s[i]=='(':
            s=s[0:i-1]
            break
    return s

df['Метро'] = df['Метро'].loc[df['Метро'].isnull()==False].apply(metro)

# Очищаем признак "Площадь, м2" от мусора

df['Площадь, м2'] = df['Площадь, м2'].astype(str).str.extract(r'(\d+\.\d+)')[0].astype(float)
df['Площадь, м2'].fillna(0)

# Очищаем признак "Цена" от мусора

df['Цена'] = df['Цена'].astype(str).str.extract(r'(\d+\.\d+)')[0].astype(float)
df['Цена'].fillna(0)

df.head(5)

,Unnamed: 0,ID объявления,Количество комнат,Метро,Время до метро пешком,"Площадь, м2",Парковка,Цена,Ремонт,Балкон,...,Можно с детьми/животными,"Высота потолков, м",Лифт,Этаж,Тип дома,Мебель в комнатах,Мебель на кухне,Холодильник,Стиральная машина,Интернет
0,0,271271157,4,Смоленская,9,200.0,подземная,500000.0,Дизайнерский,NaN,...,"Можно с детьми, Можно с животными",3.0,"Пасс (4), Груз (1)",5,Монолитный,1.0,1.0,1.0,1.0,1.0
1,1,271634126,4,Смоленская,8,198.0,подземная,500000.0,Дизайнерский,NaN,...,Можно с детьми,3.5,"Пасс (1), Груз (1)",5,Монолитно-кирпичный,1.0,1.0,1.0,1.0,1.0
2,2,271173086,4,Смоленская,7,200.0,подземная,500000.0,Евроремонт,NaN,...,Можно с детьми,3.2,Пасс (1),5,None,1.0,1.0,1.0,1.0,1.0
3,3,272197456,4,Смоленская,3,170.0,подземная,400000.0,Евроремонт,NaN,...,Можно с животными,3.2,Пасс (1),5,None,1.0,1.0,1.0,1.0,1.0
4,4,273614615,2,Арбатская,7,58.0,NaN,225000.0,Евроремонт,NaN,...,NaN,3.9,"Пасс (1), Груз (1)",12,Панельный,1.0,1.0,1.0,1.0,1.0


### Преобразование очищенных данных для работы модели ###

In [397]:
#преобразование станций в "Метро"

center_stations = (
    'Александровский сад', 'Арбатская', 'Баррикадная', 'Бауманская', 
    'Белорусская', 'Библиотека имени Ленина', 'Боровицкая', 'Бутырская',
    'Горьковская', 'Достоевская', 'Китай-город', 'Краснопресненская', 
    'Лубянка', 'Маяковская', 'Марксистская', 'Октябрьская', 'Площадь Революции',
    'Преображенская площадь', 'Революционная', 'Смоленская', 'Сретенский бульвар', 'Таганская',
    'Тверская', 'Тургеневская', 'Улица 1905 года', 'Чистые пруды'
)

mid_stations = (
    'Академическая', 'Академгородок', 'Аэропорт', 'Бабушкинская', 'Беляево', 
    'Ботанический сад', 'Варшавская', 'Водный стадион', 'Воробьевы горы', 
    'Выхино', 'Добрынинская', 'Дубровка', 'Елизаровская', 'Зеленоград', 
    'Ильинская', 'Калужская', 'Киевская', 'Кожухово', 'Коломенская', 
    'Комсомольская', 'Коньково', 'Королёв', 'Кропоткинская', 'Крымская', 
    'Кунцевская', 'Ленинский проспект', 'Лермонтовский проспект', 'Лужники',
    'Львовская', 'Люблино', 'Менделеевская', 'Молодежная', 'Мякинина',
    'Нахимовский проспект', 'Нижегородская', 'Новогиреево', 'Новослободская',
    'Новоясеневская', 'Павелецкая', 'Парк Победы', 'Перово', 'Петропавловская', 
    'Речной вокзал', 'Рязанский проспект', 'Савёловская', 'Северное Измайлово',
    'Северное Тушино', 'Семеновская', 'Софийская', 'Спортивная', 'Суходол',
    'Третьяковская', 'Трудовая', 'Трубная', 'Улица Скобелевская', 'Университет', 
    'Устинова', 'Фрунзенская', 'Хорошево', 'Цветной бульвар', 'Чертановская',
    'Шаболовская', 'Шелепиха'
)

outer_stations = (
    'Алтуфьево', 'Битцевский парк', 'Говорово', 'Деловой центр', 'Домодедовская',
    'Донской', 'Дорогомилово', 'Жулебино', 'Зябликово', 'Калужская', 'Кожухово', 
    'Коломенская', 'Комсомольская', 'Краснопресненская', 'Курская', 'Ленинский проспект',
    'Лермонтовский проспект', 'Лужники', 'Люблино', 'Марьино', 'Медведково',
    'Молодежная', 'Мякинина', 'Нахимовский проспект', 'Нижегородская', 'Новогиреево',
    'Новослободская', 'Новоясеневская', 'Павелецкая', 'Перово', 'Петропавловская',
    'Речной вокзал', 'Рязанский проспект', 'Савёловская', 'Северное Измайлово', 
    'Северное Тушино', 'Семеновская', 'Софийская', 'Спортивная', 'Суходол', 
    'Сходненская', 'Тимирязевская', 'Трубная', 'Улица Скобелевская', 'Университет', 
    'Устинова', 'Фрунзенская', 'Хорошево', 'Чертановская', 'Шаболовская', 
    'Шелепиха', 'Щелковская', 'Юго-Западная', 'Южная'
)

dict_metro = {station: 0 for station in outer_stations}
dict_metro.update({station: 1 for station in mid_stations})
dict_metro.update({station: 2 for station in center_stations})

def metro_numb(station):
    if station in dict_metro:
        return dict_metro[station]
    return None

df['Метро'] = df['Метро'].apply(metro_numb)
df['Метро'] = df['Метро'].fillna(0).astype(int)

df.head(5)

,Unnamed: 0,ID объявления,Количество комнат,Метро,Время до метро пешком,"Площадь, м2",Парковка,Цена,Ремонт,Балкон,...,Можно с детьми/животными,"Высота потолков, м",Лифт,Этаж,Тип дома,Мебель в комнатах,Мебель на кухне,Холодильник,Стиральная машина,Интернет
0,0,271271157,4,2,9,200.0,подземная,500000.0,Дизайнерский,NaN,...,"Можно с детьми, Можно с животными",3.0,"Пасс (4), Груз (1)",5,Монолитный,1.0,1.0,1.0,1.0,1.0
1,1,271634126,4,2,8,198.0,подземная,500000.0,Дизайнерский,NaN,...,Можно с детьми,3.5,"Пасс (1), Груз (1)",5,Монолитно-кирпичный,1.0,1.0,1.0,1.0,1.0
2,2,271173086,4,2,7,200.0,подземная,500000.0,Евроремонт,NaN,...,Можно с детьми,3.2,Пасс (1),5,None,1.0,1.0,1.0,1.0,1.0
3,3,272197456,4,2,3,170.0,подземная,400000.0,Евроремонт,NaN,...,Можно с животными,3.2,Пасс (1),5,None,1.0,1.0,1.0,1.0,1.0
4,4,273614615,2,2,7,58.0,NaN,225000.0,Евроремонт,NaN,...,NaN,3.9,"Пасс (1), Груз (1)",12,Панельный,1.0,1.0,1.0,1.0,1.0


In [399]:
#преобразование типов парковок в "Парковка"

df['Парковка'] = df['Парковка'].apply(lambda x: 1 if pd.notna(x) else 0)

#преобразование типов ремонтов в "Ремонт"

def repairs(r):
    d_repairs={'Без ремонта': 0,'Косметический': 1,'Евроремонт': 2, 'Дизайнерский': 3}
    if r in d_repairs.keys():
        return d_repairs[r]
    else:
        return None

df['Ремонт'] = df['Ремонт'].loc[df['Ремонт'].isnull()==False].apply(repairs)
df['Ремонт'] = df['Ремонт'].fillna(1)
df['Ремонт'] = df['Ремонт'].astype(int)

#преобразование балконов/лодж в "Балкон"

df['Балкон']= df['Балкон'].str.replace('Балкон', '1')
df['Балкон']= df['Балкон'].str.replace('Лоджия', '2')

def balc(b):
    list_numb=[]
    for i in b:
        if i.isdigit() == True:
            list_numb.append(int(i))
        else:
            continue
    if len(list_numb) > 2:
        return (list_numb[0]*list_numb[1] + list_numb[2]*list_numb[3])
    else:
        return (list_numb[0]*list_numb[1])

df['Балкон'] = df['Балкон'].loc[df['Балкон'].isnull()==False].apply(balc)
df['Балкон'] = df['Балкон'].fillna(0)
df['Балкон'] = df['Балкон'].astype(int)

#преобразование типов санузлов в "Санузел"

df['Санузел']= df['Санузел'].str.replace('Совмещенный', '1')
df['Санузел']= df['Санузел'].str.replace('Раздельный', '2')

def bath(b):
    list_numb=[]
    for i in b:
        if i.isdigit() == True:
            list_numb.append(int(i))
        else:
            continue
    if len(list_numb) > 2:
        return (list_numb[0]*list_numb[1]+list_numb[2]*list_numb[3])
    else:
        return (list_numb[0]*list_numb[1])

df['Санузел'] = df['Санузел'].loc[df['Санузел'].isnull()==False].apply(bath)
df['Санузел'] = df['Санузел'].fillna(0)
df['Санузел'] = df['Санузел'].astype(int)

#преобразование опций в "Можно с детьми/животными"

def zhiv_det(z):
    d_allow = {'Можно с детьми': 1, 'Можно с животными': 1, 'Можно с детьми, Можно с животными': 2}
    if z in d_allow.keys():
        return d_allow[z]
    else:
        return None

df['Можно с детьми/животными'] = df['Можно с детьми/животными'].loc[df['Можно с детьми/животными'].isnull()==False].apply(zhiv_det)
df['Можно с детьми/животными'] = df['Можно с детьми/животными'].fillna(0)
df['Можно с детьми/животными'] = df['Можно с детьми/животными'].astype(int)

# преобразование высоты потолка (на случий значений NaN) в "Высота потолков, м"

df['Высота потолков, м'] = df['Высота потолков, м'].astype(float)
mean_ceil = df['Высота потолков, м'].mean()
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(round(mean_ceil, 2))
df['Высота потолков, м'] = df['Высота потолков, м'].astype(float)

# преобразование наличия лифтов в "Лифт"

df['Лифт'] = df['Лифт'].apply(lambda x: 1 if pd.notna(x) else 0)

# преобразование типов дома в "Тип дома"

def houses_types(h):
    dict_types_houses = {'Панельный': 1, 'Кирпичный': 2, 'Блочный': 3, 'Монолитный': 4, 'Монолитно-кирпичный': 5, 'Сталинский': 6, 'Газобетонный': 7, 'старый': 8,
       'Щитовой': 9, 'Деревянный': 10}
    if h in dict_types_houses.keys():
        return dict_types_houses[h]
    else:
        return None

df['Тип дома'] = df['Тип дома'].loc[df['Тип дома'].isnull()==False].apply(houses_types)
df['Тип дома'] = df['Тип дома'].fillna(2)
df['Тип дома'] = df['Тип дома'].astype(int)

df.head(5)

,Unnamed: 0,ID объявления,Количество комнат,Метро,Время до метро пешком,"Площадь, м2",Парковка,Цена,Ремонт,Балкон,...,Можно с детьми/животными,"Высота потолков, м",Лифт,Этаж,Тип дома,Мебель в комнатах,Мебель на кухне,Холодильник,Стиральная машина,Интернет
0,0,271271157,4,2,9,200.0,1,500000.0,3,0,...,2,3.0,1,5,4,1.0,1.0,1.0,1.0,1.0
1,1,271634126,4,2,8,198.0,1,500000.0,3,0,...,1,3.5,1,5,5,1.0,1.0,1.0,1.0,1.0
2,2,271173086,4,2,7,200.0,1,500000.0,2,0,...,1,3.2,1,5,2,1.0,1.0,1.0,1.0,1.0
3,3,272197456,4,2,3,170.0,1,400000.0,2,0,...,1,3.2,1,5,2,1.0,1.0,1.0,1.0,1.0
4,4,273614615,2,2,7,58.0,0,225000.0,2,0,...,0,3.9,1,12,1,1.0,1.0,1.0,1.0,1.0


### Завершающий этап

In [401]:
#Переименование признаков на латиницу

df.rename(columns={'Unnamed: 0': 'Unnamed_0'}, inplace=True)
df.rename(columns={'ID  объявления': 'ID'}, inplace=True)   
df.rename(columns={'Количество комнат': 'rooms_qnty'}, inplace=True)
df.rename(columns={'Площадь, м2': 'apart_area'}, inplace=True)
df.rename(columns={'Парковка': 'parking'}, inplace=True)
df.rename(columns={'Цена': 'price'}, inplace=True)
df.rename(columns={'Ремонт': 'repairs'}, inplace=True)
df.rename(columns={'Балкон': 'balcony'}, inplace=True)
df.rename(columns={'Санузел': 'bathroom'}, inplace=True)
df.rename(columns={'Можно с детьми/животными': 'animals_kids'}, inplace=True)
df.rename(columns={'Высота потолков, м': 'ceil_height'}, inplace=True)
df.rename(columns={'Лифт': 'elevator'}, inplace=True)
df.rename(columns={'Метро': 'metro_station_rating'}, inplace=True)
df.rename(columns={'Время до метро пешком': 'time_to_metro'}, inplace=True)
df.rename(columns={'Этаж': 'floor'}, inplace=True)
df.rename(columns={'Тип дома': 'house_type'}, inplace=True)
df.rename(columns={'Мебель в комнатах': 'room_furn'}, inplace=True)
df.rename(columns={'Мебель на кухне': 'kitch_furn'}, inplace=True)
df.rename(columns={'Холодильник': 'fridge'}, inplace=True)
df.rename(columns={'Стиральная машина': 'washing_mach'}, inplace=True)
df.rename(columns={'Интернет': 'internet'}, inplace=True)


#Упорядочивание признаков - в том порядке, в каком у всех участников команды

new_order = ['Unnamed_0', 'ID', 'rooms_qnty', 'metro_station_rating',
             'time_to_metro', 'apart_area', 'floor', 'house_type', 'parking',
             'price', 'repairs', 'balcony', 'bathroom', 'animals_kids', 'room_furn',
             'kitch_furn', 'fridge', 'washing_mach', 'internet', 'ceil_height', 'elevator']

df = df.loc[:, ~df.columns.duplicated()]

df = df.reindex(columns=new_order)

df

,Unnamed_0,ID,rooms_qnty,metro_station_rating,time_to_metro,apart_area,floor,house_type,parking,price,...,balcony,bathroom,animals_kids,room_furn,kitch_furn,fridge,washing_mach,internet,ceil_height,elevator
0,0,271271157,4,2,9,200.0,5,4,1,500000.0,...,0,0,2,1.0,1.0,1.0,1.0,1.0,3.00,1
1,1,271634126,4,2,8,198.0,5,5,1,500000.0,...,0,4,1,1.0,1.0,1.0,1.0,1.0,3.50,1
2,2,271173086,4,2,7,200.0,5,2,1,500000.0,...,0,3,1,1.0,1.0,1.0,1.0,1.0,3.20,1
3,3,272197456,4,2,3,170.0,5,2,1,400000.0,...,0,3,1,1.0,1.0,1.0,1.0,1.0,3.20,1
4,4,273614615,2,2,7,58.0,12,1,0,225000.0,...,0,2,0,1.0,1.0,1.0,1.0,1.0,3.90,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,23363,215565511,1,0,8,35.0,10,2,0,42000.0,...,1,1,0,1.0,1.0,1.0,1.0,1.0,2.98,1
23364,23364,274654844,1,0,7,38.7,5,4,0,45000.0,...,2,1,0,1.0,1.0,1.0,1.0,0.0,2.98,1
23365,23365,268679909,2,0,6,43.1,5,2,0,50000.0,...,1,1,1,0.0,1.0,1.0,1.0,1.0,2.98,0
23366,23366,274807525,2,0,11,52.5,8,4,1,55000.0,...,2,3,0,1.0,1.0,1.0,1.0,0.0,2.65,1


In [369]:
df.to_csv('Marina_half_data.csv', index=False)